In [1]:
import os
from pathlib import Path
from torch.utils.data import DataLoader
from mp_transformer.config import CONFIG
from mp_transformer.train import setup

In [2]:
current_dir = Path.cwd().parts[-1]
if current_dir == "demo":
    os.chdir("..")
!pwd

/data/daniel/git/mp-transformer


In [3]:
CONFIG["hidden_dim"] = 40
CONFIG["latent_dim"] = 48
CONFIG["num_primitives"] = 1
CONFIG["num_attention_heads"] = 1
CONFIG["num_transformer_layers"] = 1
model_vae_single_layer_transformer, _, test_set = setup(CONFIG, test_set=True)
model_vae_single_layer_transformer = model_vae_single_layer_transformer.load_from_checkpoint("demo/BEST-VAE-single-layer-transformer.ckpt", config=CONFIG)

In [4]:
CONFIG["hidden_dim"] = 40
CONFIG["latent_dim"] = 48
CONFIG["num_primitives"] = 1
CONFIG["num_attention_heads"] = 4
CONFIG["num_transformer_layers"] = 4
model_vae, _, _ = setup(CONFIG)
model_vae = model_vae.load_from_checkpoint("demo/BEST-VAE.ckpt", config=CONFIG)

In [5]:
test_dataloader = DataLoader(
    test_set,
    batch_size=CONFIG["batch_size"],
    drop_last=True,
    num_workers=4,
)

In [6]:
CONFIG["hidden_dim"] = 40
CONFIG["latent_dim"] = 8
CONFIG["num_primitives"] = 6
CONFIG["num_attention_heads"] = 1
CONFIG["num_transformer_layers"] = 1
model_single_layer_transformer, _, _ = setup(CONFIG)
model_single_layer_transformer = model_single_layer_transformer.load_from_checkpoint("demo/BEST-single-layer-transformer.ckpt", config=CONFIG)

In [7]:
CONFIG["hidden_dim"] = 40
CONFIG["latent_dim"] = 8
CONFIG["num_primitives"] = 6
CONFIG["num_attention_heads"] = 4
CONFIG["num_transformer_layers"] = 4
model, _, _ = setup(CONFIG)
model = model.load_from_checkpoint("demo/BEST-bottleneck.ckpt", config=CONFIG)

In [8]:
from pytorch_lightning import Trainer

trainer = Trainer(gpus=1)
res = trainer.test(model, dataloaders=test_dataloader)
res

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 39/39 [00:00<00:00, 41.19it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      avg_test_loss        0.011915095150470734
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'avg_test_loss': 0.011915095150470734}]